# Types of Module
PowerShell modules are collections of functionality that can be exported and imported as needed. Modules allow for code encapsulation, 



There are three types of PowerShell module:
- **Script Modules:** A script module is a PowerShell script file with the extension `.psm1`. This extension enables the use of module cmdlets (such as `Import-Module` on the file. Script modules must be stored in a directory with the same name as the `.psm1` script itself.
- **Binary Modules:** A binary module is a DLL containing PowerShell functionality written in a managed language such as C#. Even a simple .NET cmdlet assembly is treated as a binary module with no additional work, however the cmdlet assembly can be additional packaged up with other resources and a manifest file in order to distribute larger or more complex projects. Note: Binary modules will not be demonstrated in this notebook. An example of how to write a cmdlet assembly can be found in the [Binary Cmdlets](./10.%20Binary%20Cmdlets.ipynb) notebook.
- **Manifest Modules:** A manifest module is a module that only contains a manifest file (with the `.psd1` extension as mentioned above) and doesn't contain any sort of functionality contained in an assembly or script. A manifest module as a convenient way to package up resources that other modules will use, such as nested modules, assemblies, and types.

There are also _CIM_ and _Workflow_ module types, but they are not covered here.

The `Get-Module` cmdlet below retrieves all the modules loaded into the current session.

In [1]:
Get-Module


ModuleType Version    PreRelease Name                                ExportedCommands
---------- -------    ---------- ----                                ----------------
Manifest   7.0.0.0               Microsoft.PowerShell.Management     {Add-Content, Clear-Content, …



# Script Modules
The code below is a trivial example of how to write, export and import a script module. The here-string stored in `$scriptModule` contains a function that will be the core of the example script module, and also contains a call to the `Export-ModuleMember` cmdlet, which to exports the function. This is saved into a `.psm1` file, which turns the code into a fully fledged scipt module.

In [2]:
$scriptModule = @' 
    function Show-Users {
        Get-CimInstance -ClassName Win32_Account | Where-Object { $_.SIDType -eq 1 } | Select-Object Caption, SID
    }
    Export-ModuleMember -Function Show-Users
'@
$scriptModule | Out-File -FilePath .\exampleScriptModule.psm1
"This is the script module we are going to import:"
Get-ChildItem ./e*

This is the script module we are going to import:

    Directory: C:\Users\alex\Projects\ALiterateTourOfPowerShell

Mode                 LastWriteTime         Length Name
----                 -------------         ------ ----
-a---           6/21/2024  1:14 PM            192 exampleScriptModule.psm1



The function contained in the module is now member function of that module, and when the module is imported using the `Import-Module` cmdlet, it is this member function that is exported and loaded into the session. Trying to use the function before the module is loaded fails as expected.

In [3]:
try {
    Show-Users | Format-Table -Autosize
} catch {
    "`nNope, as expected we hit an exception as module isnt loaded yet!"
}


Nope, as expected we hit an exception as module isnt loaded yet!


Error: Command failed: SubmitCode: try { ...

Now we will import the module using the appropriately named `Import-Module` cmdlet and check its loaded using `Get-Module` cmdlet.

The exported member of the module is the command Show-Users, and now the module is loadaed this function should be callable. Running it shows the SIDs for local user accounts.

In [53]:
Import-Module ./exampleScriptModule.psm1
Get-Module -Name exampleScriptModule
Show-Users | Format-Table
Remove-Module exampleScriptModule # to clean up


ModuleType Version    PreRelease Name                                ExportedCommands
---------- -------    ---------- ----                                ----------------
Script     0.0                   exampleScriptModule                 Show-Users


Caption                            SID
-------                            ---
DESKTOP-TM5O5E8\Administrator      S-1-5-21-2776884319-3090142823-3378317822-500
DESKTOP-TM5O5E8\alex               S-1-5-21-2776884319-3090142823-3378317822-1001
DESKTOP-TM5O5E8\DefaultAccount     S-1-5-21-2776884319-3090142823-3378317822-503
DESKTOP-TM5O5E8\Guest              S-1-5-21-2776884319-3090142823-3378317822-501
DESKTOP-TM5O5E8\WDAGUtilityAccount S-1-5-21-2776884319-3090142823-3378317822-504



# Manifests and Manifest Modules
An important component of a PowerShell module is the _manifest_, which is file with the `.psd1` extension that is used to describe the contents of the module, its requirements and dependencies, as well as define exactly the functionality that the module exports. Manifests are an optional, but powerful feature of modules.

In [43]:
$manifestContents = @{
    Path = './exampleManifest.psd1' 
    RootModule = './exampleScriptModule.psm1' 
    Author = 'hashford'
}
New-ModuleManifest @manifestContents
Test-ModuleManifest -Path ./exampleManifest.psd1


ModuleType Version    PreRelease Name                                ExportedCommands
---------- -------    ---------- ----                                ----------------
Script     0.0.1                 exampleManifest                     Show-Users



As can be seen below the Show-Users function us currently unavailable, as the script module `exampleScriptModule` was unloaded.

In [55]:
Show-Users

Show-Users: The term 'Show-Users' is not recognized as a name of a cmdlet, function, script file, or executable program.
Check the spelling of the name, or if a path was included, verify that the path is correct and try again.


Error: Command failed: SubmitCode: Show-Users

In [54]:
Import-Module .\exampleManifest.psd1
Get-Module -Name exampleManifest | Select Name, RootModule, Path, Author | Format-List
Show-Users
Remove-Module exampleManifest # to clean up


Name       : exampleManifest
RootModule : ./exampleScriptModule.psm1
Path       : C:\Users\alex\Projects\ALiterateTourOfPowerShell\exampleScriptModule.psm1
Author     : hashford


Caption                            SID
-------                            ---
DESKTOP-TM5O5E8\Administrator      S-1-5-21-2776884319-3090142823-3378317822-500
DESKTOP-TM5O5E8\alex               S-1-5-21-2776884319-3090142823-3378317822-1001
DESKTOP-TM5O5E8\DefaultAccount     S-1-5-21-2776884319-3090142823-3378317822-503
DESKTOP-TM5O5E8\Guest              S-1-5-21-2776884319-3090142823-3378317822-501
DESKTOP-TM5O5E8\WDAGUtilityAccount S-1-5-21-2776884319-3090142823-3378317822-504



# A Note on PSModulePath

# Dynamic Modules
Finally it is also possible to create a _dynamic module_ completely in memory, to use within the session it is declared. This achieved via the `New-Module` cmdlet:

In [51]:
New-Module -ScriptBlock {
    function GetCodePIDs {
        Get-Process -name code | Select-Object Id, StartTime
    }
}
Get-Module -Name GetCodePIDs
GetCodePIDs | Format-Table -AutoSize


ModuleType Version    PreRelease Name                                ExportedCommands
---------- -------    ---------- ----                                ----------------
Script     0.0                   __DynamicModule_152d5ac5-46c5-4b9b… GetCodePIDs


   Id StartTime
   -- ---------
 3084 6/21/2024 12:30:24 PM
 3728 6/21/2024 12:30:27 PM
 4404 6/21/2024 12:30:26 PM
 6564 6/21/2024 12:30:28 PM
 9116 6/21/2024 12:30:24 PM
 9380 6/21/2024 12:30:28 PM
 9528 6/21/2024 12:30:24 PM
 9768 6/21/2024 12:30:24 PM
11232 6/21/2024 12:48:41 PM
11444 6/21/2024 12:30:23 PM

